Импорт библиотек

In [1]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import scipy.stats as st
import seaborn as sns

from permute.core import two_sample
from arch.bootstrap import IIDBootstrap

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

Загрузка данных

In [2]:
data = pd.read_csv('./nba.csv')

Выборка представляет собой статистику бросков для различных игроков NBA

In [3]:
data.head()

,Unnamed: 0,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
0,7,A. Johnson,ATL,30,34,24,28,64,52
1,10,A. Johnson,TOR,63,57,44,38,120,82
2,15,Abdur-Rahim,SAC,114,124,84,90,238,174
3,22,Afflalo,DEN,236,284,189,227,520,416
4,23,Afflalo,DET,75,88,62,67,163,129


Требуется определить, есть ли разница в успехе бросков у игроков в домашних и гостевых играх.

Определим для каждого игрока успех (долю) попадания в играх дома и в гостях (делаем потому, что количество бросков дома и в гостях разное - биномиальный критерий использовать некорректно)

In [4]:
atm_away = data.loc[:, 'atm_away'].values
atm_home = data.loc[:, 'atm_home'].values
score_away = data.loc[:, 'score_away'].values
score_home = data.loc[:, 'score_home'].values

x_away = []
for i in range(0, len(score_away)):
    x_away.append(score_away[i] / atm_away[i])
    
x_home = []
for i in range(0, len(score_home)):
    x_home.append(score_home[i] / atm_home[i])

# Критерий

Теперь мы имеем две связанные выборки (каждые доли попадания в играх дома и в гостях связаны тем, что броски были сделаны одним игроком)

Проверим гипотезу $H_0$: процент успешных бросков не зависит от места проведения матча (средние значения связанных выборок равны)

Альтернативная гипотеза $H_1$: средние значения связанных выборок НЕ равны

Для проверки нулевой гипотезы воспользуемся двухвыборочным перестановочным критерием для связанных выборок

In [5]:
p_value = two_sample(x_away, x_home,  alternative='two-sided')[0]
p_value

0.50064

p-value $\geqslant 0.05$, следовательно данные гипотезе не противоречат

# Бутстреповый доверительный интервал

Построим доверительный интервал для средней разности с помощью бутстрепа:

In [6]:
delta = score_away / atm_away - score_home / atm_home
bs = IIDBootstrap(delta)
bs.conf_int(np.mean, 100000, method='basic', size=0.95)

array([[-0.00831975],
       [ 0.00154743]])

Средние значения выборок равны $\Leftrightarrow$ среднее значение разности равно 0.

Видим, что 0 попадает в доверительный интервал.

# Вывод

Из результатов перестановочного критерия и метода доверительных интервалов с помощью бутстрепа можно заключить, что данные не противоречат гипотезе о независимости успешности бросков от места проведения матча на уровне значимости 0.05